In [1]:
import numpy as np
import pandas as pd
import random
from copy import deepcopy

In [2]:
df = []

size = 1e4

A = ['a0', 'a1']
B = ['b0', 'b1', 'b2', 'b3', 'b4']
C = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
D = ['d0']

for i in range(int(size)):
    df.append({'A': random.choice(A), 'B': random.choice(B), 'C': random.choice(C), 'D': D[0]})

df = pd.DataFrame(df)
df['A'] = df['A'].astype('category')
df['B'] = df['B'].astype('category')
df['C'] = df['C'].astype('category')
df['D'] = df['D'].astype('category')
df

,A,B,C,D
0,a0,b2,c9,d0
1,a0,b4,c2,d0
2,a0,b1,c4,d0
3,a1,b0,c7,d0
4,a0,b3,c8,d0
...,...,...,...,...
9995,a0,b0,c5,d0
9996,a1,b3,c8,d0
9997,a1,b3,c7,d0
9998,a0,b3,c3,d0


### "Element-based" implementation of block representation (correct)

In [101]:
def init_state(s):
    categories = list(s.cat.categories)
    
    # decide the number of parts
    n_parts = random.choice(range(1, len(categories)+1))
    if n_parts == 1:
        return [categories]
    elif n_parts == len(categories):
        return [[x] for x in categories]
    else:
        container = []
        # decide the part elements
        random.shuffle(categories)
        ix_bars = sorted(random.sample(range(1, len(categories)), k=n_parts-1))
        ix_bars = [0] + ix_bars + [len(categories)]
        container = []
        for i, ix in enumerate(ix_bars):
            if i > 0:
                container.append(categories[ix_bars[i-1]:ix])
        return container

In [106]:
pA = init_state(df['A'])
print(pA)
print(len(pA))
pB = init_state(df['B'])
print(pB)
print(len(pB))
pC = init_state(df['C'])
print(pC)
print(len(pC))
pD = init_state(df['D'])
print(pD)
print(len(pD))

[['a0'], ['a1']]
2
[['b1'], ['b3'], ['b4', 'b2', 'b0']]
3
[['c0'], ['c1', 'c3'], ['c4', 'c9', 'c5'], ['c7', 'c2', 'c6', 'c8']]
4
[['d0']]
1


In [133]:
def split(container, max_n):
    if len(container) == max_n:
        return None
    else:
        new_container = container.copy()
        indices = list(range(len(new_container)))
        random.shuffle(indices)
        for i in indices:
            if len(new_container[i]) > 1:
                # only split a part that are not singleton
                ix_bar = 1 if len(new_container[i]) == 2 else random.choice(range(1, len(new_container[i])))
                random.shuffle(new_container[i])
                new_container.append(new_container[i][:ix_bar])
                new_container.append(new_container[i][ix_bar:])
                # append does not change existing indices, pop safely
                new_container.pop(i)
                return new_container

In [148]:
ret = pB
while True:
    ret = split(ret, 5)
    print(ret)
    if ret is None:
        break
print(pB)

[['b1'], ['b3'], ['b2'], ['b4', 'b0']]
[['b1'], ['b3'], ['b2'], ['b0'], ['b4']]
None
[['b1'], ['b3'], ['b2', 'b4', 'b0']]


In [119]:
def merge(container):
    if len(container) == 1:
        return None
    elif len(container) == 2:
        return [container[0] + container[1]]
    else:
        new_container = []
        ixs_to_merge = random.sample(range(len(container)), k=2)
        merged = container[ixs_to_merge[0]].copy() + container[ixs_to_merge[1]].copy()
        for i in range(len(container)):
            if i not in ixs_to_merge:
                new_container.append(container[i].copy())
        new_container.append(merged)
        return new_container

In [124]:
ret = pB
while True:
    ret = merge(ret)
    print(ret)
    if ret is None:
        break
print(pB)

[['b3'], ['b4', 'b2', 'b0', 'b1']]
[['b3', 'b4', 'b2', 'b0', 'b1']]
None
[['b1'], ['b3'], ['b4', 'b2', 'b0']]


In [152]:
%timeit split([[f'x{i}' for i in range(100)]], 100)

204 µs ± 549 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [153]:
%timeit merge([[f'x{i}'] for i in range(100)])

157 µs ± 841 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### "Membership-based" implementation of graphical representation (correct)

In [302]:
def init_state3(s):
    categories = sorted(s.cat.categories)
    n = len(categories)
    
    if n == 1:
        return np.array([[1]], dtype=int)
    
    mat = np.zeros((n, n), dtype=int)
    for i, x in enumerate(categories):
        i_part = np.random.choice(len(categories))
        mat[i, i_part] = 1
    
    return mat

In [319]:
pA = init_state3(df['A'])
print(pA)
print(len(pA))
pB = init_state3(df['B'])
print(pB)
print(len(pB))
pC = init_state3(df['C'])
print(pC)
print(len(pC))
pD = init_state3(df['D'])
print(pD)
print(len(pD))

[[0 1]
 [0 1]]
2
[[0 0 1 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]]
5
[[0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]]
10
[[1]]
1


In [320]:
def identify(mat):
    base = 2 ** np.array(range(len(mat)))
    dot_prod = np.dot(base, mat)
    return frozenset(dot_prod[dot_prod != 0])
    
#     s = set()
#     for arr in mat[:,cols_nonzero].T.tolist():
#         print(''.join(str(ele) for ele in arr))
        

In [321]:
print(identify(pA))
print(identify(pB))
print(identify(pC))
print(identify(pD))

frozenset({3})
frozenset({8, 17, 2, 4})
frozenset({64, 1, 32, 256, 512, 134, 8, 16})
frozenset({1})


In [256]:
def split3(mat):
    if np.all(np.any(mat, axis=0)) and np.sum(mat) == len(mat):
        # all singletons
        return None
    mat = mat.copy()
    rng = np.random.default_rng()
    arr_sum = np.sum(mat, axis=0)
    cols_nonzero = np.unique(np.nonzero(mat)[1])
    first_col_allzero = np.amin(np.nonzero(np.all(mat == 0, axis=0))[0])
    np.random.shuffle(cols_nonzero)
    for col in cols_nonzero:
        if np.sum(mat[:,col]) > 1:
            rows_nonzero = np.nonzero(mat[:,col])[0]
            i_bar = rng.choice(len(rows_nonzero) - 1) + 1
            mat[rows_nonzero[i_bar:],col] = 0
            mat[rows_nonzero[i_bar:],first_col_allzero] = 1
            return mat

In [263]:
ret = pB
while True:
    ret = split3(ret)
    print(ret)
    if ret is None:
        break
print(pB)

[[0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]
[[0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]]
None
[[0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [287]:
def merge3(mat):
    if np.sum(np.all(mat, axis=0)) == 1:
        # only singleton
        return None
    mat = mat.copy()
    rng = np.random.default_rng()
    cols_nonzero = np.unique(np.nonzero(mat)[1])
    if len(cols_nonzero) > 2:
        sel_cols_nonzero = cols_nonzero[rng.choice(len(cols_nonzero), size=2)]
    else:
        sel_cols_nonzero = cols_nonzero[:]
    merged_col = mat[:,sel_cols_nonzero[0]] + mat[:,sel_cols_nonzero[1]]
    mat[:,sel_cols_nonzero[0]] = merged_col
    mat[:,sel_cols_nonzero[1]] = 0
    return mat    

In [325]:
ret = pB
while True:
    ret = merge3(ret)
    print(ret)
    if ret is None:
        break
print(pB)
print(np.nonzero(np.dot(pB, np.array([0, 0, 1, 0, 0]).T)))

[[0 1 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 1 0 0 0]]
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [1 0 0 0 0]]
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]]
None
[[0 0 1 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]]
(array([0, 4]),)


In [290]:
x = np.zeros((100, 100))
x[:0] = 1
%timeit split3(x)

678 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [291]:
x = np.eye(100)
%timeit merge3(x)

774 µs ± 5.78 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### "Index-based" implementation of block representation (incorrect)

In [11]:
def init_state2(s):
    categories = s.cat.categories
    idx = list(range(len(categories)))
    #random.shuffle(idx)
    # decide the number of parts
    n_parts = random.choice(range(1, len(categories)+1))
    ix_bars = sorted(random.sample(range(1, len(idx)), k=n_parts-1))
    return idx, ix_bars

def split2(ix_bars, max_n):
    if len(ix_bars) + 1 == max_n:
        return None
    else:
        ix_bars_vacant = set(range(1, max_n)) - set(ix_bars)
        ix_bar_add = random.choice(list(ix_bars_vacant))
        return sorted(ix_bars + [ix_bar_add])
    
def merge2(ix_bars):
    if len(ix_bars) == 0:
        return None
    elif len(ix_bars) == 1:
        return []
    else:
        iix_bar_rm = random.choice(range(len(ix_bars)))
        new_ix_bars = ix_bars.copy()
        new_ix_bars.pop(iix_bar_rm)
        return new_ix_bars
    

In [12]:
idxA, barsA = init_state2(df['A'])
print((idxA, barsA))
idxB, barsB = init_state2(df['B'])
print((idxB, barsB))
idxC, barsC = init_state2(df['C'])
print((idxC, barsC))
idxD, barsD = init_state2(df['D'])
print((idxD, barsD))

([0, 1], [])
([0, 1, 2, 3, 4], [])
([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [2, 3])
([0], [])


In [13]:
ret = barsC
while True:
    ret = split2(ret, 10)
    print(ret)
    if ret is None:
        break
print(barsC)

[1, 2, 3]
[1, 2, 3, 5]
[1, 2, 3, 5, 7]
[1, 2, 3, 5, 6, 7]
[1, 2, 3, 5, 6, 7, 8]
[1, 2, 3, 5, 6, 7, 8, 9]
[1, 2, 3, 4, 5, 6, 7, 8, 9]
None
[2, 3]


In [14]:
ret = barsC
while True:
    ret = merge2(ret)
    print(ret)
    if ret is None:
        break
print(barsC)

[3]
[]
None
[2, 3]


In [29]:
%timeit split2([], 100)

18.6 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [30]:
%timeit merge2(list(range(1, 100)))

7.59 µs ± 44.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
